In [1]:
# 导入必要的库
import sys
import os

# 添加项目根目录到Python路径
sys.path.insert(0, os.path.abspath('../../'))

# 导入数据库相关模块
from table.TJCAUSP import TJCAUSP
from sql.database import get_db_session, get_db
import pandas as pd

In [4]:
# 连接数据库并查询TJCAUSP表的数据
with get_db_session() as db:
    records = db.query(TJCAUSP).all()

    # 将查询结果转换为pandas DataFrame
    data = []
    for record in records:
        data.append({
            'id': record.id,
            'ORDER_NO': record.ORDER_NO,
            'IN_MAT_WIDTH': record.IN_MAT_WIDTH,
            'IN_MAT_THICK': record.IN_MAT_THICK
        })
    df = pd.DataFrame(data)
    print(f"从TJCAUSP_ORIGIN表中查询到 {len(df)} 条记录")

    print(df.head(10))

2025-10-22 13:13:30,885 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-22 13:13:30,886 INFO sqlalchemy.engine.Engine SELECT "TJCAUSP".id AS "TJCAUSP_id", "TJCAUSP"."ORDER_NO" AS "TJCAUSP_ORDER_NO", "TJCAUSP"."ORDER_MONTH" AS "TJCAUSP_ORDER_MONTH", "TJCAUSP"."IN_MAT_WIDTH" AS "TJCAUSP_IN_MAT_WIDTH", "TJCAUSP"."IN_MAT_THICK" AS "TJCAUSP_IN_MAT_THICK" 
FROM "TJCAUSP"
2025-10-22 13:13:30,888 INFO sqlalchemy.engine.Engine [cached since 170.3s ago] ()
从TJCAUSP_ORIGIN表中查询到 106 条记录
   id    ORDER_NO  IN_MAT_WIDTH  IN_MAT_THICK
0   1  G025000870    885.751638      2.038891
1   2  G025000870    883.453724      2.113444
2   3  G025000870    887.162539      2.098953
3   4  G025000870    881.116535      2.021794
4   5  G025000870    889.916988      2.026222
5   6  G025000870    866.940810      0.213000
6   7  G025000870    864.694310      0.216890
7   8  G025000870    865.764242      0.217971
8   9  G025000870    865.829424      0.218385
9  10  G025000870    865.544815      0.216681
2025-10

In [68]:

def plot_feature_sequence(feature, order, feature_name="宽度"):
    """
    使用 Plotly 绘制宽度序列
    :param feature: 原始属性
    :param order: 最优顺序索引
    :param feature_name: 属性
    """
    title="序列" + feature_name + "分布图"
    ordered_feature = feature[order]
    positions = list(range(len(feature)))

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=positions,
        y=ordered_feature,
        mode='lines+markers+text',
        textposition="top center",
        line=dict(color='steelblue', width=2),
        marker=dict(size=8, color='steelblue')
    ))

    fig.update_layout(
        title=title,
        xaxis_title="序列顺序",
        yaxis_title=feature_name,
        template="plotly_white"
    )

    fig.show()


In [329]:
# 目标函数
def objective(order, width, thick):
    """
    目标函数
    :param order: 板坯序列索引
    :param width: 原始宽度数组
    :param thick: 原始厚度数组
    """
    thick_cost= objective_thick(order, thick)

    width_cost = objective_width_advanced(order, width)

    return 100 * thick_cost + width_cost


# 这里的目标函数不仅仅是计算两者的相邻差分，现在是这样：如果相邻板坯，前者比后者宽，则此时cost就是相邻差分的平方和；但是如果前者比比后者窄，我们将其称为返宽，此时是一个比相邻差分平方还要大的cost，意味着我们不想让他反宽；但是如果当前反宽很大，我们又想让他存在，相当于另起一个序列，这样我们放弃了当前的序列，我们就可以让从这个新的材料的后续序列能够继续慢慢递减下去，所以这里的cost反而要设计的小一点，让其有可能被选中

# 宽度
def objective_width(order, width):
    w = width[order]
    diffs = np.abs(np.diff(w))
    mean_d = np.mean(diffs)
    # 希望相邻差分接近平均值（越均匀越好）
    return np.sum((diffs - mean_d)**2)

def objective_width_advanced(order, width, small_penalty=100000, large_penalty=0.00001, k=2.0):
    """
    :param order: 板坯序列索引
    :param width: 原始宽度数组
    :param small_penalty: 小反宽惩罚系数
    :param large_penalty: 大反宽惩罚系数
    :param k: 阈值灵敏度（用于判定大反宽）
    """

    #  静态宽度正跳惩罚表
    static_cost = {"0-10": 0}


    cost = 0
    threshold = 150

    for i in range(len(order) - 1):
        index = order[i]
        next_index = order[i+1]

        cur_width = width[index]
        next_width = width[next_index]

        diff = cur_width - next_width

        if index == 0:
            if diff < 0:
                cost += 100000

        if diff > 0:
            if 0 <= diff <= 10:
                cost += static_cost["0-10"]
            else:
                cost += diff * 2
        else:
            if diff > threshold:
                cost += 3000
            else:
                cost += 150 - (abs(diff) - threshold) / 10
    return cost


# 厚度
def objective_thick(order, thick):

    max_diff = np.max(thick) - np.min(thick)

    cost = 0

    for i in range(len(order) - 1):
        index = order[i]
        next_index = order[i+1]

        cur_thick = thick[index]
        next_thick = thick[next_index]

        diff = abs(cur_thick - next_thick)

        normal_diff = diff / max_diff * 20

        cost += normal_diff
    return cost



In [326]:
# 局部搜索
import math
import numpy as np
import random
import plotly.graph_objects as go


def local_search(width, thick, max_iter = 20000):

    n = len(width)
    best_order = np.argsort(-width)  # 初始解：严格递减
    best_cost = objective(best_order, width, thick)
    print("best_cost: ", best_cost)


    for _ in range(max_iter):
        i, j = random.sample(range(n), 2)
        new_order = best_order.copy()
        new_order[i], new_order[j] = new_order[j], new_order[i]

        new_cost = objective(new_order, width, thick)
        if new_cost < best_cost:
            print("iteration: ", _, "best_cost: ", new_cost)
            best_order, best_cost = new_order, new_cost

    return best_order, best_cost

def simulated_annealing(width, thick, max_iter=20000, T0=1.0, alpha=0.995):
    n = len(width)
    current = np.argsort(-width)
    current_cost = objective(current, width, thick)
    best = current.copy()
    best_cost = current_cost
    T = T0  # 初始温度

    for k in range(max_iter):
        i, j = random.sample(range(n), 2)
        new = current.copy()
        new[i], new[j] = new[j], new[i]
        new_cost = objective(new, width, thick)
        delta = new_cost - current_cost

        # 关键：以一定概率接受更差的解
        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = new, new_cost
            if new_cost < best_cost:
                print("iteration: ", _, "best_cost: ", new_cost)
                best, best_cost = new.copy(), new_cost

        T *= alpha  # 降温

    return best, best_cost

def local_search_width(width, max_iter=20000):
    n = len(width)
    best_order = np.argsort(-width)  # 初始解：严格递减
    best_cost = objective_width_advanced(best_order, width)

    for _ in range(max_iter):
        i, j = random.sample(range(n), 2)
        new_order = best_order.copy()
        new_order[i], new_order[j] = new_order[j], new_order[i]

        # 检查反宽数量是否 <= 3
        # if np.sum(np.diff(width[new_order]) > 0) <= 3:
        new_cost = objective_width_advanced(new_order, width)

        if new_cost < best_cost:
            best_order, best_cost = new_order, new_cost

    return best_order, best_cost

def local_search_thick(thick, max_iter=20000):
    n = len(thick)
    best_order = np.argsort(thick)
    best_cost = objective_thick(best_order, thick)

    for _ in range(max_iter):
        i, j = random.sample(range(n), 2)
        new_order = best_order.copy()
        new_order[i], new_order[j] = new_order[j], new_order[i]

        new_cost = objective_thick(new_order, thick)
        if new_cost < best_cost:
            best_order, best_cost = new_order, new_cost

    return best_order, best_cost

In [330]:

width = df["IN_MAT_WIDTH"].values
thick = df["IN_MAT_THICK"].values

original_order = np.arange(len(df))
plot_feature_sequence(width, original_order, "宽度")
plot_feature_sequence(thick, original_order, "厚度")

start_order = np.argsort(-width)
plot_feature_sequence(width, start_order, "宽度")
plot_feature_sequence(thick, start_order, "厚度")

best_order, best_cost = simulated_annealing(width, thick, max_iter=100000)

# 结果后处理，再加个启发式，把宽度突出的序列找到最近的厚度相邻那里再重新计算值

print("最优顺序：")
print(best_order)
print(df.iloc[best_order])
print("目标函数值：", best_cost)

plot_feature_sequence(width, best_order, "宽度")
plot_feature_sequence(thick, best_order, "厚度")

iteration:   best_cost:  22997.15839692094
iteration:   best_cost:  19808.799480516154
iteration:   best_cost:  19713.672454755215
iteration:   best_cost:  19710.929307052433
iteration:   best_cost:  19706.103748793845
iteration:   best_cost:  19651.575297152813
iteration:   best_cost:  19602.05635417447
iteration:   best_cost:  16857.544333241785
iteration:   best_cost:  16854.960705795347
iteration:   best_cost:  16854.925997449285
iteration:   best_cost:  16844.875830521698
iteration:   best_cost:  16826.95385592613
iteration:   best_cost:  16812.59957038152
iteration:   best_cost:  16647.792654811677
iteration:   best_cost:  16645.181173266945
iteration:   best_cost:  16641.620520732242
iteration:   best_cost:  16582.048107999097
iteration:   best_cost:  16582.048107999093
iteration:   best_cost:  16516.907624762614
iteration:   best_cost:  16516.90762476261
iteration:   best_cost:  16508.927273321104
iteration:   best_cost:  16507.053572096436
iteration:   best_cost:  16438.118316